In [1]:
import os, sys
import itertools
import numpy as np
from tqdm.notebook import trange

# image
import cv2
import glob
from skimage.io import imread, imsave, imshow
from PIL import Image, ImageTk
from imutils import paths
import matplotlib.pyplot as plt 
import itertools
import numpy as np
from tqdm.notebook import trange, tqdm_notebook

# image
import cv2
from skimage.io import imread, imsave, imshow
from PIL import Image, ImageTk
from imutils import paths
import matplotlib.pyplot as plt

# customized function
from core.imageprep import dir_checker, random_crop, crop_generator, random_crop_batch
from core.imageprep import create_crop_idx, crop_to_patch, construct_from_patch, create_crop_idx_whole

In [2]:
# on mac
main_path = "/Volumes/LaCie_DataStorage/PerlmutterData/"
img_folder = "segmentation_compiled_export"
folder_raw = "raw_img"

ip_path = os.path.join(main_path, img_folder, 'input')
dir_checker('output', os.path.join(main_path, img_folder))

rawflist = glob.glob(os.path.join(ip_path, folder_raw, '*'))
print(rawflist)

foldernames = [os.path.basename(x) for x in rawflist]
print(foldernames)

labelname = {
    1: 'cell_membrane',
    2: 'nucleus',
    3: 'nucleolus',
    4: 'lipid_droplet',
    5: 'mito',
    6: 'inclusion',
    7: 'ER',
    8: 'cristae',
}
print(labelname)

output exists in /Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export
['/Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export/input/raw_img/data_d00_batch01_loc01', '/Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export/input/raw_img/data_d00_batch02_loc02', '/Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export/input/raw_img/data_d00_batch02_loc03', '/Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export/input/raw_img/data_d07_batch01_loc01', '/Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export/input/raw_img/data_d07_batch02_loc01', '/Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export/input/raw_img/data_d07_batch02_loc02', '/Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export/input/raw_img/data_d14_batch01_loc01', '/Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export/input/raw_img/data_d17_batch01_loc01', '/Volumes/LaCie_DataSto

In [3]:
len(labelname)

8

In [11]:
class sample: 
    def __init__(self, foldername):
        self.foldername = os.path.basename(foldername)
    def merge(self, path, labels):
        print(self.foldername)
        print(path)
        
        # set up path
        filelist = {}
        filelist['raw_img'] = glob.glob(os.path.join(path, 'input', 'raw_img', self.foldername, '*.tif'))

        new_labels = {}
        for key, item in labels.items():
            filelist_tmp = glob.glob(os.path.join(path, 'input', item, self.foldername, '*.tif*'))
            if len(filelist_tmp) != 0:
                filelist[item] = filelist_tmp 
                new_labels[key] = item

        img_tmp = imread(filelist['raw_img'][0])
        img_y, img_x = img_tmp.shape

        dir_checker(self.foldername, os.path.join(path, 'output'))
        
        for idx in tqdm_notebook(range(len(filelist['raw_img'])), leave=False):
            img_tmp_base = np.zeros([img_y, img_x, len(labels)], dtype= 'uint8')
            
            for j, label in new_labels.items():
                open_filename = os.path.join(path, 'input', label, self.foldername, filelist[label][idx])
                img_open = imread(open_filename)
                img_open = img_open.astype('uint8')
                img_tmp_base[:, :, j-1] = img_open * (j)
            
            img_tmp_result = np.amax(img_tmp_base, axis=2)
            imsave(os.path.join(path, 'output', self.foldername, str(idx).zfill(4) + '.tif'), img_tmp_result)


In [12]:
for foldername in rawflist: 
    folder_tmp = sample(foldername)
    print(folder_tmp.foldername)
    folder_tmp.merge(os.path.join(main_path, img_folder), labelname)

data_d00_batch01_loc01
data_d00_batch01_loc01
/Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export
data_d00_batch01_loc01 exists in /Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export/output


data_d00_batch02_loc02
data_d00_batch02_loc02
/Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export
data_d00_batch02_loc02 does not exist in /Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export/output


data_d00_batch02_loc03
data_d00_batch02_loc03
/Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export
data_d00_batch02_loc03 does not exist in /Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export/output


data_d07_batch01_loc01
data_d07_batch01_loc01
/Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export
data_d07_batch01_loc01 does not exist in /Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export/output


data_d07_batch02_loc01
data_d07_batch02_loc01
/Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export
data_d07_batch02_loc01 does not exist in /Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export/output


data_d07_batch02_loc02
data_d07_batch02_loc02
/Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export
data_d07_batch02_loc02 does not exist in /Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export/output


data_d14_batch01_loc01
data_d14_batch01_loc01
/Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export
data_d14_batch01_loc01 does not exist in /Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export/output


data_d17_batch01_loc01
data_d17_batch01_loc01
/Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export
data_d17_batch01_loc01 does not exist in /Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export/output


data_d21_batch01_loc01
data_d21_batch01_loc01
/Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export
data_d21_batch01_loc01 exists in /Volumes/LaCie_DataStorage/PerlmutterData/segmentation_compiled_export/output
